<a href="https://colab.research.google.com/github/SYWoo02/Engineer_BigData_Analysis_10/blob/main/Part2_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이진분류 연습문제

## Section 2 이직 여부 예측
### 새로운 일자리를 찾을지 예측하시오.
### 학습용 데이터(train)를 이용해 새 일자리를 찾을지 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.
* pred: 예측값(이직할 확률)
* 제출 파일명: 'result.csv'
* 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점한다.

In [11]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
test = pd.read_csv("hr_test.csv")
train = pd.read_csv("hr_train.csv")

In [12]:
# EDA
print("===== Train 데이터 확인 =====")
print(train.head(5))
print("\n")

print("===== Test 데이터 확인 =====")
print(test.head(5))
print("\n")

print("===== 데이터 형태 확인 =====")
print("train:", train.shape)
print("test:", test.shape)
print("\n")

print("===== 데이터 타입 확인 =====")
print(train.info())
print("\n")

print("===== 데이터 결측치 여부 확인 =====")
print("train:\n", train.isnull().sum())
print("test:\n", test.isnull().sum())
print("\n")

print("===== target 빈도 =====")
print(train['target'].value_counts())

===== Train 데이터 확인 =====
   enrollee_id      city  city_development_index gender  \
0        30266   city_84                   0.698   Male   
1        13254   city_16                   0.910   Male   
2        31675   city_21                   0.624   Male   
3        30804  city_104                   0.924   Male   
4        18269  city_143                   0.740   Male   

       relevent_experience enrolled_university education_level  \
0   No relevent experience    Full time course     High School   
1  Has relevent experience       no_enrollment        Graduate   
2   No relevent experience    Full time course     High School   
3   No relevent experience       no_enrollment        Graduate   
4  Has relevent experience       no_enrollment        Graduate   

  major_discipline experience company_size company_type last_new_job  \
0              NaN          1      100-500      Pvt Ltd        never   
1             STEM         15        50-99      Pvt Ltd            1   
2      

In [13]:
# train과 test 범주형 데이터 비교
cols = train.select_dtypes(include='object').columns
for col in cols:
  set_train = set(train[col])
  set_test = set(test[col])
  same = (set_train == set_test)
  if same:
    print(col,"\t카테고리 동일함")
  else:
    print(col,"\t카테고리 동일하지 않음")

city 	카테고리 동일하지 않음
gender 	카테고리 동일함
relevent_experience 	카테고리 동일함
enrolled_university 	카테고리 동일함
education_level 	카테고리 동일함
major_discipline 	카테고리 동일함
experience 	카테고리 동일함
company_size 	카테고리 동일함
company_type 	카테고리 동일함
last_new_job 	카테고리 동일함


In [14]:
# train 데이터에서 target 분리
target = train.pop('target')

In [15]:
# 결측치 처리
train = train.fillna("X")
test = test.fillna("X")

In [17]:
# train과 test 합쳐서 원-핫 인코딩
combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)
n_train = len(train)
train = combined_dummies[:n_train]
test = combined_dummies[n_train:]

# Validation 나누기
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.3, random_state = 0)

# 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print("roc_auc:", roc_auc)

roc_auc: 0.7885760304052094


In [20]:
# 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv",index=False)

In [21]:
# 생성한 결과 확인
result = pd.read_csv("result.csv")
result.head(5)

,pred
0,0.24
1,0.43
2,0.45
3,0.02
4,0.03
